In [31]:
import pandas as pd 
import numpy as np
import datetime
from pathlib import Path
import os

pd.set_option('display.max_columns', 500)


In [13]:
data_path = Path(os.getcwd()) / 'data' / 'WEC2022_Data'

In [14]:
data_path

PosixPath('/home/kuba1302/UW/wec_2022/data/WEC2022_Data')

In [17]:
df = pd.read_csv(data_path / 'BKG_train.csv', sep=';')

In [24]:
df

,BOOKING_ID,SALES_DATE,SALES_MARKET,SALES_CHANNEL,TRIP_TYPE,BOOKING_WINDOW_D,STAY_LENGTH_D,BOOKING_LONG_HOUL_FLAG,BOOKING_DOMESTIC_FLAG,FLIGHT_COUPONS,SEGMENTS,PAX_N,INTINERARY,BOOKING_ORIGIN_AIRPORT,BOOKING_ORIGIN_COUNTRY_CODE,BOOKING_DEPARTURE_TIME_UTC,BOOKING_DESTINATION_AIRPORT,BOOKING_DESTINATION_COUNTRY_CODE,BOOKING_ARRIVAL_TIME_UTC,UPGRADED_FLAG,UPGRADE_SALES_DATE,UPGRADE_TYPE
0,2754285175826930,2007-01-26,3823004278339643,AGENTS,ROUND TRIP,61.0,9.0,N,N,2,2,1,LFF-GMW-LFF,LFF,EE,2007-03-28 04:00:00,GMW,DE,2007-04-06 21:10:00,N,NaN,NaN
1,667762693252160,2007-10-05,6982998843464221,AGENTS,ROUND TRIP,10.0,4.0,Y,N,4,2,1,VMY-HIX-OUO-OIH-OUO-HIX-VMY,HIX,US,2007-10-15 22:25:00,OIH,PL,2007-10-19 00:35:00,N,NaN,NaN
2,7185592089077946,2007-01-08,3823004278339643,AGENTS,ONE WAY,38.0,-9999.0,N,N,1,1,1,WXA-OUO,WXA,FR,2007-02-15 06:30:00,OUO,PL,2007-02-15 08:45:00,N,NaN,NaN
3,4685868825417887,2007-11-14,2460355646543430,AGENTS,ONE WAY,16.0,-9999.0,N,N,1,1,1,UGK-OUO,UGK,NL,2007-11-30 18:55:00,OUO,PL,2007-11-30 20:55:00,N,NaN,NaN
4,6948280039050277,2007-09-26,4802659914165873,AGENTS,ROUND TRIP,29.0,4.0,N,N,8,2,2,EDN-OUO-FOH-OUO-EDN,EDN,MD,2007-10-25 14:55:00,FOH,UA,2007-10-29 12:05:00,N,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795935,8277851166644641,2007-10-08,4391133056062766,LOT.COM,ONE WAY,6.0,-9999.0,N,N,4,1,2,FWU-OUO-KNH,FWU,CY,2007-10-14 00:40:00,KNH,RU,2007-10-14 10:50:00,N,NaN,NaN
2795936,6336772439230224,2007-04-26,8557423498357146,LOT.COM,ROUND TRIP,19.0,1.0,N,N,4,2,1,JHS-OUO-FBI-OUO-JHS,JHS,PL,2007-05-15 12:55:00,FBI,GB,2007-05-16 21:50:00,N,NaN,NaN
2795937,2491938906106596,2007-01-08,6745023307045216,AGENTS,ROUND TRIP,10.0,3.0,N,N,2,2,1,FBI-OUO-FBI,FBI,GB,2007-01-18 10:25:00,OUO,PL,2007-01-21 17:20:00,N,NaN,NaN
2795938,4105882171748456,2007-07-16,4802659914165873,AGENTS,MULTICITY,14.0,1.0,N,N,2,2,1,FBI-OUO-ELO,OUO,PL,2007-07-30 08:45:00,NaN,NaN,2007-07-31 08:00:00,N,NaN,NaN


In [21]:
tkt = pd.read_csv(data_path / 'TKT_train.csv', sep=';')

In [58]:
fcp = pd.read_csv(data_path / 'FCP_train.csv', sep=';')
emd = pd.read_csv(data_path / 'EMD_train.csv', sep=';')


In [ ]:
emd

Error: Session cannot generate requests

In [22]:
tkt

,BOOKING_ID,TICKET_NUMBER,ORIGINAL_TICKET_NUMBER,FORM_OF_PAYMENT,CURRENCY,TOTAL_PRICE,TOTAL_PRICE_PLN,PAX_GENDER,PAX_TYPE,CORPORATE_CONTRACT_FLG,LOYAL_CUSTOMER,LOYAL_CUSTOMER_ID,LOYAL_CUSTOMER_DATE_OF_BIRTH,LOYAL_CUSTOMER_REGISTERED_DATE
0,4443948267837742,2433160688245721,9379265972706784,CCCA5,EUR,564.14,2350.94,M,ADULT,N,N,NaN,NaN,NaN
1,8827981391920441,2285471225231829,9379265972706784,CA,PLN,470.50,470.50,M,ADULT,N,N,NaN,NaN,NaN
2,7756899638586197,8700163489079041,9379265972706784,CA,PLN,218.49,218.49,M,ADULT,N,N,NaN,NaN,NaN
3,6411974034589912,1412002991872234,9379265972706784,CA,PLN,720.61,720.61,M,ADULT,N,N,NaN,NaN,NaN
4,6730427035328795,1715275704997737,9379265972706784,CCCA5,PLN,415.61,415.61,M,ADULT,N,N,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3944498,7532867952732900,7387270018960696,9379265972706784,CCCA5,SEK,2395.00,998.24,M,ADULT,N,Y,4.904725e+15,1968-12-07,2003-05-07
3944499,4390923384627090,4028422919052082,9379265972706784,CA,EUR,109.17,467.83,M,ADULT,N,N,NaN,NaN,NaN
3944500,9268232053474716,6586299214365763,9379265972706784,CCVI4,USD,532.90,2007.33,M,ADULT,N,N,NaN,NaN,NaN
3944501,451171036064944,233248134999409,9379265972706784,CCTP1,USD,692.50,2602.97,M,ADULT,N,N,NaN,NaN,NaN


In [27]:
df

,BOOKING_ID,SALES_DATE,SALES_MARKET,SALES_CHANNEL,TRIP_TYPE,BOOKING_WINDOW_D,STAY_LENGTH_D,BOOKING_LONG_HOUL_FLAG,BOOKING_DOMESTIC_FLAG,FLIGHT_COUPONS,SEGMENTS,PAX_N,INTINERARY,BOOKING_ORIGIN_AIRPORT,BOOKING_ORIGIN_COUNTRY_CODE,BOOKING_DEPARTURE_TIME_UTC,BOOKING_DESTINATION_AIRPORT,BOOKING_DESTINATION_COUNTRY_CODE,BOOKING_ARRIVAL_TIME_UTC,UPGRADED_FLAG,UPGRADE_SALES_DATE,UPGRADE_TYPE
0,2754285175826930,2007-01-26,3823004278339643,AGENTS,ROUND TRIP,61.0,9.0,N,N,2,2,1,LFF-GMW-LFF,LFF,EE,2007-03-28 04:00:00,GMW,DE,2007-04-06 21:10:00,N,NaN,NaN
1,667762693252160,2007-10-05,6982998843464221,AGENTS,ROUND TRIP,10.0,4.0,Y,N,4,2,1,VMY-HIX-OUO-OIH-OUO-HIX-VMY,HIX,US,2007-10-15 22:25:00,OIH,PL,2007-10-19 00:35:00,N,NaN,NaN
2,7185592089077946,2007-01-08,3823004278339643,AGENTS,ONE WAY,38.0,-9999.0,N,N,1,1,1,WXA-OUO,WXA,FR,2007-02-15 06:30:00,OUO,PL,2007-02-15 08:45:00,N,NaN,NaN
3,4685868825417887,2007-11-14,2460355646543430,AGENTS,ONE WAY,16.0,-9999.0,N,N,1,1,1,UGK-OUO,UGK,NL,2007-11-30 18:55:00,OUO,PL,2007-11-30 20:55:00,N,NaN,NaN
4,6948280039050277,2007-09-26,4802659914165873,AGENTS,ROUND TRIP,29.0,4.0,N,N,8,2,2,EDN-OUO-FOH-OUO-EDN,EDN,MD,2007-10-25 14:55:00,FOH,UA,2007-10-29 12:05:00,N,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795935,8277851166644641,2007-10-08,4391133056062766,LOT.COM,ONE WAY,6.0,-9999.0,N,N,4,1,2,FWU-OUO-KNH,FWU,CY,2007-10-14 00:40:00,KNH,RU,2007-10-14 10:50:00,N,NaN,NaN
2795936,6336772439230224,2007-04-26,8557423498357146,LOT.COM,ROUND TRIP,19.0,1.0,N,N,4,2,1,JHS-OUO-FBI-OUO-JHS,JHS,PL,2007-05-15 12:55:00,FBI,GB,2007-05-16 21:50:00,N,NaN,NaN
2795937,2491938906106596,2007-01-08,6745023307045216,AGENTS,ROUND TRIP,10.0,3.0,N,N,2,2,1,FBI-OUO-FBI,FBI,GB,2007-01-18 10:25:00,OUO,PL,2007-01-21 17:20:00,N,NaN,NaN
2795938,4105882171748456,2007-07-16,4802659914165873,AGENTS,MULTICITY,14.0,1.0,N,N,2,2,1,FBI-OUO-ELO,OUO,PL,2007-07-30 08:45:00,NaN,NaN,2007-07-31 08:00:00,N,NaN,NaN


In [28]:
stay_lenght_map = {-9999: None}

In [40]:
df['STAY_LENGTH_D'] = df['STAY_LENGTH_D'].replace(stay_lenght_map)
df['BOOKING_ARRIVAL_TIME_UTC'] = pd.to_datetime(df['BOOKING_ARRIVAL_TIME_UTC'])
df['BOOKING_DEPARTURE_TIME_UTC'] = pd.to_datetime(df['BOOKING_DEPARTURE_TIME_UTC'])

In [52]:
df['flight_len'] = df['BOOKING_ARRIVAL_TIME_UTC'] - df['BOOKING_DEPARTURE_TIME_UTC']
df['flight_len'] = df['flight_len'].apply(lambda x: x.seconds / 3600)

In [ ]:
bkg_drop_cols = ['UPGRADED_FLAG', 'UPGRADED_TYPE', 'UPGRADE_SALES_DATE']
df = df.drop(columns=bkg_drop_cols)

In [41]:
test_1 = df.iloc[0]

In [48]:
(test_1['BOOKING_ARRIVAL_TIME_UTC'] - test_1['BOOKING_DEPARTURE_TIME_UTC']).seconds / 3600

17.166666666666668

In [53]:
df

,BOOKING_ID,SALES_DATE,SALES_MARKET,SALES_CHANNEL,TRIP_TYPE,BOOKING_WINDOW_D,STAY_LENGTH_D,BOOKING_LONG_HOUL_FLAG,BOOKING_DOMESTIC_FLAG,FLIGHT_COUPONS,SEGMENTS,PAX_N,INTINERARY,BOOKING_ORIGIN_AIRPORT,BOOKING_ORIGIN_COUNTRY_CODE,BOOKING_DEPARTURE_TIME_UTC,BOOKING_DESTINATION_AIRPORT,BOOKING_DESTINATION_COUNTRY_CODE,BOOKING_ARRIVAL_TIME_UTC,UPGRADED_FLAG,UPGRADE_SALES_DATE,UPGRADE_TYPE,flight_len
0,2754285175826930,2007-01-26,3823004278339643,AGENTS,ROUND TRIP,61.0,9.0,N,N,2,2,1,LFF-GMW-LFF,LFF,EE,2007-03-28 04:00:00,GMW,DE,2007-04-06 21:10:00,N,NaN,NaN,17.166667
1,667762693252160,2007-10-05,6982998843464221,AGENTS,ROUND TRIP,10.0,4.0,Y,N,4,2,1,VMY-HIX-OUO-OIH-OUO-HIX-VMY,HIX,US,2007-10-15 22:25:00,OIH,PL,2007-10-19 00:35:00,N,NaN,NaN,2.166667
2,7185592089077946,2007-01-08,3823004278339643,AGENTS,ONE WAY,38.0,-9999.0,N,N,1,1,1,WXA-OUO,WXA,FR,2007-02-15 06:30:00,OUO,PL,2007-02-15 08:45:00,N,NaN,NaN,2.250000
3,4685868825417887,2007-11-14,2460355646543430,AGENTS,ONE WAY,16.0,-9999.0,N,N,1,1,1,UGK-OUO,UGK,NL,2007-11-30 18:55:00,OUO,PL,2007-11-30 20:55:00,N,NaN,NaN,2.000000
4,6948280039050277,2007-09-26,4802659914165873,AGENTS,ROUND TRIP,29.0,4.0,N,N,8,2,2,EDN-OUO-FOH-OUO-EDN,EDN,MD,2007-10-25 14:55:00,FOH,UA,2007-10-29 12:05:00,N,NaN,NaN,21.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795935,8277851166644641,2007-10-08,4391133056062766,LOT.COM,ONE WAY,6.0,-9999.0,N,N,4,1,2,FWU-OUO-KNH,FWU,CY,2007-10-14 00:40:00,KNH,RU,2007-10-14 10:50:00,N,NaN,NaN,10.166667
2795936,6336772439230224,2007-04-26,8557423498357146,LOT.COM,ROUND TRIP,19.0,1.0,N,N,4,2,1,JHS-OUO-FBI-OUO-JHS,JHS,PL,2007-05-15 12:55:00,FBI,GB,2007-05-16 21:50:00,N,NaN,NaN,8.916667
2795937,2491938906106596,2007-01-08,6745023307045216,AGENTS,ROUND TRIP,10.0,3.0,N,N,2,2,1,FBI-OUO-FBI,FBI,GB,2007-01-18 10:25:00,OUO,PL,2007-01-21 17:20:00,N,NaN,NaN,6.916667
2795938,4105882171748456,2007-07-16,4802659914165873,AGENTS,MULTICITY,14.0,1.0,N,N,2,2,1,FBI-OUO-ELO,OUO,PL,2007-07-30 08:45:00,NaN,NaN,2007-07-31 08:00:00,N,NaN,NaN,23.250000


In [55]:
tkt

,BOOKING_ID,TICKET_NUMBER,ORIGINAL_TICKET_NUMBER,FORM_OF_PAYMENT,CURRENCY,TOTAL_PRICE,TOTAL_PRICE_PLN,PAX_GENDER,PAX_TYPE,CORPORATE_CONTRACT_FLG,LOYAL_CUSTOMER,LOYAL_CUSTOMER_ID,LOYAL_CUSTOMER_DATE_OF_BIRTH,LOYAL_CUSTOMER_REGISTERED_DATE
0,4443948267837742,2433160688245721,9379265972706784,CCCA5,EUR,564.14,2350.94,M,ADULT,N,N,NaN,NaN,NaN
1,8827981391920441,2285471225231829,9379265972706784,CA,PLN,470.50,470.50,M,ADULT,N,N,NaN,NaN,NaN
2,7756899638586197,8700163489079041,9379265972706784,CA,PLN,218.49,218.49,M,ADULT,N,N,NaN,NaN,NaN
3,6411974034589912,1412002991872234,9379265972706784,CA,PLN,720.61,720.61,M,ADULT,N,N,NaN,NaN,NaN
4,6730427035328795,1715275704997737,9379265972706784,CCCA5,PLN,415.61,415.61,M,ADULT,N,N,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3944498,7532867952732900,7387270018960696,9379265972706784,CCCA5,SEK,2395.00,998.24,M,ADULT,N,Y,4.904725e+15,1968-12-07,2003-05-07
3944499,4390923384627090,4028422919052082,9379265972706784,CA,EUR,109.17,467.83,M,ADULT,N,N,NaN,NaN,NaN
3944500,9268232053474716,6586299214365763,9379265972706784,CCVI4,USD,532.90,2007.33,M,ADULT,N,N,NaN,NaN,NaN
3944501,451171036064944,233248134999409,9379265972706784,CCTP1,USD,692.50,2602.97,M,ADULT,N,N,NaN,NaN,NaN


In [56]:
df = df.merge(tkt, on='BOOKING_ID')

In [57]:
df

,BOOKING_ID,SALES_DATE,SALES_MARKET,SALES_CHANNEL,TRIP_TYPE,BOOKING_WINDOW_D,STAY_LENGTH_D,BOOKING_LONG_HOUL_FLAG,BOOKING_DOMESTIC_FLAG,FLIGHT_COUPONS,SEGMENTS,PAX_N,INTINERARY,BOOKING_ORIGIN_AIRPORT,BOOKING_ORIGIN_COUNTRY_CODE,BOOKING_DEPARTURE_TIME_UTC,BOOKING_DESTINATION_AIRPORT,BOOKING_DESTINATION_COUNTRY_CODE,BOOKING_ARRIVAL_TIME_UTC,UPGRADED_FLAG,UPGRADE_SALES_DATE,UPGRADE_TYPE,flight_len,TICKET_NUMBER,ORIGINAL_TICKET_NUMBER,FORM_OF_PAYMENT,CURRENCY,TOTAL_PRICE,TOTAL_PRICE_PLN,PAX_GENDER,PAX_TYPE,CORPORATE_CONTRACT_FLG,LOYAL_CUSTOMER,LOYAL_CUSTOMER_ID,LOYAL_CUSTOMER_DATE_OF_BIRTH,LOYAL_CUSTOMER_REGISTERED_DATE
0,2754285175826930,2007-01-26,3823004278339643,AGENTS,ROUND TRIP,61.0,9.0,N,N,2,2,1,LFF-GMW-LFF,LFF,EE,2007-03-28 04:00:00,GMW,DE,2007-04-06 21:10:00,N,NaN,NaN,17.166667,6161890315396952,9379265972706784,CCCA5,SEK,1962.00,829.14,F,ADULT,N,N,NaN,NaN,NaN
1,667762693252160,2007-10-05,6982998843464221,AGENTS,ROUND TRIP,10.0,4.0,Y,N,4,2,1,VMY-HIX-OUO-OIH-OUO-HIX-VMY,HIX,US,2007-10-15 22:25:00,OIH,PL,2007-10-19 00:35:00,N,NaN,NaN,2.166667,2792860646521702,9379265972706784,CCAX3,USD,1553.73,5828.35,M,ADULT,N,N,NaN,NaN,NaN
2,7185592089077946,2007-01-08,3823004278339643,AGENTS,ONE WAY,38.0,-9999.0,N,N,1,1,1,WXA-OUO,WXA,FR,2007-02-15 06:30:00,OUO,PL,2007-02-15 08:45:00,N,NaN,NaN,2.250000,9751558490165889,9379265972706784,CCCA5,SEK,883.00,373.33,F,ADULT,N,N,NaN,NaN,NaN
3,4685868825417887,2007-11-14,2460355646543430,AGENTS,ONE WAY,16.0,-9999.0,N,N,1,1,1,UGK-OUO,UGK,NL,2007-11-30 18:55:00,OUO,PL,2007-11-30 20:55:00,N,NaN,NaN,2.000000,5261470551404339,9379265972706784,CA,PLN,1316.64,1316.64,M,ADULT,N,N,NaN,NaN,NaN
4,6948280039050277,2007-09-26,4802659914165873,AGENTS,ROUND TRIP,29.0,4.0,N,N,8,2,2,EDN-OUO-FOH-OUO-EDN,EDN,MD,2007-10-25 14:55:00,FOH,UA,2007-10-29 12:05:00,N,NaN,NaN,21.166667,1506317744159641,9379265972706784,CCCA5,EUR,175.34,752.89,M,ADULT,N,N,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3944498,8277851166644641,2007-10-08,4391133056062766,LOT.COM,ONE WAY,6.0,-9999.0,N,N,4,1,2,FWU-OUO-KNH,FWU,CY,2007-10-14 00:40:00,KNH,RU,2007-10-14 10:50:00,N,NaN,NaN,10.166667,5370887325315325,9379265972706784,CCVI4,EUR,308.25,1329.08,F,ADULT,N,Y,6.412769e+15,1959-08-15,2004-08-31
3944499,6336772439230224,2007-04-26,8557423498357146,LOT.COM,ROUND TRIP,19.0,1.0,N,N,4,2,1,JHS-OUO-FBI-OUO-JHS,JHS,PL,2007-05-15 12:55:00,FBI,GB,2007-05-16 21:50:00,N,NaN,NaN,8.916667,8920416922703749,9379265972706784,CCVI4,EUR,285.51,1203.68,M,ADULT,N,Y,6.188254e+15,1951-11-17,1991-11-30
3944500,2491938906106596,2007-01-08,6745023307045216,AGENTS,ROUND TRIP,10.0,3.0,N,N,2,2,1,FBI-OUO-FBI,FBI,GB,2007-01-18 10:25:00,OUO,PL,2007-01-21 17:20:00,N,NaN,NaN,6.916667,53327094570569,9379265972706784,CCVI4,GBP,184.30,860.00,M,ADULT,N,N,NaN,NaN,NaN
3944501,4105882171748456,2007-07-16,4802659914165873,AGENTS,MULTICITY,14.0,1.0,N,N,2,2,1,FBI-OUO-ELO,OUO,PL,2007-07-30 08:45:00,NaN,NaN,2007-07-31 08:00:00,N,NaN,NaN,23.250000,5528446127148732,9379265972706784,CCCA5,EUR,194.56,842.48,M,ADULT,N,N,NaN,NaN,NaN
